In [1]:
!pip install --upgrade pyarrow
!pip install duckdb
!pip install --upgrade pandas==2.2.2 numpy==2.0.2

In [2]:
import os
import json
from collections import defaultdict
import pandas as pd
import pickle
from collections import defaultdict
import pyarrow.parquet as pq
import pyarrow as pa
import json
import pyarrow.dataset as ds
import json
from pathlib import Path
import duckdb

In [3]:
main_dir = Path("/lustre/groups/itg/shared/referenceData/OpenTargets/")

base_path = main_dir / "data_version_29_07"
output_path = main_dir / "temp/01_filtered_parquets"
manifest_path = main_dir / "temp/columns_manifest.json"

output_path.mkdir(parents=True, exist_ok=True)

with manifest_path.open() as f:
    manifest = json.load(f)

In [ ]:
for dataset_name, columns in manifest.items():
    dataset_dir = base_path / dataset_name
    try:

        if not dataset_dir.exists():
            print(f"⚠️ Directory does not exist: {dataset_dir}")
            continue
            
        dataset = ds.dataset(
            dataset_dir, 
            format="parquet", 
            exclude_invalid_files=True
        )
        
        schema = dataset.schema
        available_columns = [field.name for field in schema]
        print(f" {dataset_name} available columns: {available_columns}")
        
        valid_columns = [col for col in columns if col in available_columns]
        missing_columns = [col for col in columns if col not in available_columns]
        
        if missing_columns:
            print(f"⚠️ {dataset_name} missing columns: {missing_columns}")
        
        if valid_columns:
            table = dataset.to_table(columns=valid_columns)
            output_file = os.path.join(output_path, f"{dataset_name}.parquet")
            pq.write_table(table, output_file)
            print(f"✅ Saved {dataset_name} with {len(table)} rows and columns: {valid_columns}")
        else:
            print(f"❌ No valid columns found for {dataset_name}")
            
    except Exception as e:
        print(f"❌ Failed to process {dataset_name}: {e}")

print(f"\n Processing complete! Filtered datasets saved to: {output_path}")

 colocalisation_coloc available columns: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'rightStudyType', 'numberColocalisingVariants', 'h0', 'h1', 'h2', 'h3', 'h4', 'colocalisationMethod', 'betaRatioSignAverage']
✅ Saved colocalisation_coloc with 38561709 rows and columns: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4']
 credible_set available columns: ['studyLocusId', 'studyId', 'variantId', 'chromosome', 'position', 'region', 'beta', 'zScore', 'pValueMantissa', 'pValueExponent', 'effectAlleleFrequencyFromSource', 'standardError', 'subStudyDescription', 'qualityControls', 'finemappingMethod', 'credibleSetIndex', 'credibleSetlog10BF', 'purityMeanR2', 'purityMinR2', 'locusStart', 'locusEnd', 'sampleSize', 'ldSet', 'locus', 'confidence', 'studyType', 'isTransQtl']
✅ Saved credible_set with 2833758 rows and columns: ['studyLocusId', 'studyId']
 l2g_prediction available columns: ['studyLocusId', 'geneId', 'score', 'features', 'shapBaseValue']
✅ Saved l2g_prediction 

In [ ]:
def analyze_gwas_colocalizations(output_path: str) -> tuple:
    """
    Analyze GWAS colocalizations and create drug-target-disease associations.
    
    This function performs a two-step analysis of genetic colocalization data:
    1. Identifies high-confidence GWAS colocalizations with gene targets
    2. Links these colocalizations to known drugs and diseases
    
    Parameters
    ----------
    output_path : str
        Path to directory containing filtered OpenTargets parquet files:
        - colocalisation_coloc.parquet
        - credible_set.parquet  
        - l2g_prediction.parquet
        - study.parquet
        - known_drug.parquet
        - disease.parquet
    
    Returns
    -------
    tuple[pd.DataFrame, pd.DataFrame]
        - df_gwas: GWAS colocalizations with gene targets
        - df_final: Complete dataset with drug-disease associations
    
    Notes
    -----
    Query filters:
    - h4 > 0.8: High-confidence colocalizations only
    - studyId LIKE 'GCST%': GWAS Catalog studies only
    - Sample size: 10,000 rows for memory efficiency
    - Requires non-null gene predictions (l2g.geneId)
    
    The analysis links:
    - Colocalizations → Study loci → Genes → Drugs → Diseases
    
    Examples
    --------
    >>> df_gwas, df_final = analyze_gwas_colocalizations('/path/to/data')
    >>> print(f"Found {len(df_final)} gene-drug-disease associations")
    """
    
    # Initialize DuckDB connection
    con = duckdb.connect()
    
    print("🧬 GWAS-focused analysis")
    
    # GWAS colocalizations with gene targets
    gwas_focused_query = f"""
    SELECT 
      coloc.leftStudyLocusId,      -- Unique ID for left study locus in colocalization
      coloc.rightStudyLocusId,     -- Unique ID for right study locus in colocalization  
      coloc.chromosome,            -- Chromosome where colocalization occurs
      coloc.h4,                    -- Posterior probability of shared causal variant (0-1)
      credible.studyId,            -- GWAS study identifier (e.g., GCST90...)
      l2g.geneId AS targetId,      -- Ensembl gene ID predicted to be affected
      l2g.score AS l2g_score,      -- Locus-to-gene prediction confidence (0-1)
      study.studyType,             -- Type of study (gwas, qtl, etc.)
      study.traitFromSource,       -- Human-readable trait description
      study.projectId              -- Source project (e.g., FINNGEN, UKB)
    FROM (
      SELECT * FROM read_parquet('{output_path}/colocalisation_coloc.parquet')
      WHERE h4 > 0.8               -- High-confidence colocalizations only
      USING SAMPLE 10000 ROWS      -- Memory-efficient sampling
    ) coloc
    LEFT JOIN read_parquet('{output_path}/credible_set.parquet') credible
      ON coloc.leftStudyLocusId = credible.studyLocusId
    LEFT JOIN read_parquet('{output_path}/l2g_prediction.parquet') l2g
      ON credible.studyLocusId = l2g.studyLocusId
    LEFT JOIN read_parquet('{output_path}/study.parquet') study
      ON credible.studyId = study.studyId
    WHERE l2g.geneId IS NOT NULL   -- Must have gene prediction
      AND credible.studyId LIKE 'GCST%'  -- GWAS Catalog studies only
    ORDER BY coloc.h4 DESC, l2g.score DESC
    LIMIT 100
    """
    
    try:
        df_gwas = con.execute(gwas_focused_query).df()
        print(f"✅ GWAS-focused dataset: {len(df_gwas)} rows")
        print(f"📊 Unique genes: {df_gwas['targetId'].nunique()}")
        print(f"📚 Unique GWAS studies: {df_gwas['studyId'].nunique()}")
        
        print("\n📋 Columns:", df_gwas.columns.tolist())
        print("\n🔬 Top GWAS colocalizations:")
        print(df_gwas[['chromosome', 'h4', 'targetId', 'studyId', 'traitFromSource']].head())
        
    except Exception as e:
        print(f"❌ GWAS query failed: {e}")
        return None, None

    print("\n💊 Final query with drugs:")

    # Step 2: Add drug and disease information
    final_query = f"""
    SELECT 
      coloc.leftStudyLocusId,       -- Colocalization identifiers
      coloc.rightStudyLocusId, 
      coloc.chromosome,             -- Genomic location
      coloc.h4,                     -- Colocalization confidence
      credible.studyId,             -- GWAS study ID
      l2g.geneId AS targetId,       -- Drug target gene
      kd.diseaseId,                 -- Disease ontology ID (e.g., Orphanet_...)
      kd.drugId,                    -- ChEMBL drug identifier
      d.name AS diseaseName,        -- Human-readable disease name
      study.traitFromSource,        -- GWAS trait description
      study.studyType,              -- Study type
      l2g.score AS l2g_score        -- Gene prediction confidence
    FROM (
      SELECT * FROM read_parquet('{output_path}/colocalisation_coloc.parquet')
      WHERE h4 > 0.8
      USING SAMPLE 10000 ROWS
    ) coloc
    LEFT JOIN read_parquet('{output_path}/credible_set.parquet') credible
      ON coloc.leftStudyLocusId = credible.studyLocusId
    LEFT JOIN read_parquet('{output_path}/l2g_prediction.parquet') l2g
      ON credible.studyLocusId = l2g.studyLocusId
    LEFT JOIN read_parquet('{output_path}/study.parquet') study
      ON credible.studyId = study.studyId
    LEFT JOIN read_parquet('{output_path}/known_drug.parquet') kd
      ON l2g.geneId = kd.targetId   -- Link genes to drugs
    LEFT JOIN read_parquet('{output_path}/disease.parquet') d
      ON kd.diseaseId = d.id        -- Add disease names
    WHERE l2g.geneId IS NOT NULL
      AND credible.studyId LIKE 'GCST%'
    ORDER BY coloc.h4 DESC, l2g.score DESC
    LIMIT 100
    """

    try:
        df_final = con.execute(final_query).df()
        print(f"✅ Final dataset: {len(df_final)} rows")
        
        # Data quality assessment
        print("\n📊 Data completeness:")
        key_columns = ['targetId', 'diseaseId', 'drugId', 'diseaseName', 'traitFromSource']
        for col in key_columns:
            if col in df_final.columns:
                non_null = df_final[col].notna().sum()
                print(f"  {col}: {non_null}/{len(df_final)} ({non_null/len(df_final)*100:.1f}%)")
        
        # Validate expected schema
        print("\n🔍 DataFrame structure validation:")
        desired_columns = ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4', 
                          'studyId', 'targetId', 'diseaseId', 'drugId', 'diseaseName']
        
        for col in desired_columns:
            if col in df_final.columns:
                print(f"✅ {col}")
            else:
                print(f"❌ {col} (missing)")
        
        # Preview results
        print("\n👀 Sample of final DataFrame:")
        preview_cols = ['chromosome', 'h4', 'targetId', 'traitFromSource', 'drugId', 'diseaseName']
        print(df_final[preview_cols].head())
        
        # Highlight actionable results
        complete_rows = df_final[df_final['drugId'].notna()]
        if len(complete_rows) > 0:
            print(f"\n🎯 {len(complete_rows)} rows have complete drug information!")
            drug_summary_cols = ['targetId', 'traitFromSource', 'drugId', 'diseaseName']
            print(complete_rows[drug_summary_cols].head())
        else:
            print("\n⚠️  No complete drug information in this sample")
        
        return df_gwas, df_final
        
    except Exception as e:
        print(f"❌ Final query failed: {e}")
        return df_gwas, None
        
    finally:
        # Clean up database connection
        con.close()


if __name__ == "__main__":
    """
    Example usage of the GWAS colocalization analysis.
    
    This script generates two datasets:
    1. High-confidence GWAS colocalizations with gene targets
    2. Complete gene-drug-disease association network
    
    Results can be used for:
    - Drug repurposing opportunities
    - Target validation
    - Precision medicine biomarkers
    """
    
    # Run analysis
    df_gwas, df_final = analyze_gwas_colocalizations(output_path)
    
    if df_final is not None:
        print("\n🎉 Analysis complete!")
        print(f"📊 Generated {len(df_final)} gene-drug-disease associations")
        print("💡 Use df_final for downstream drug repurposing analysis")

        df_final.to_csv(main_dir / "colocalisation_drug_association_snapshot.csv", index=False)

🧬 GWAS-focused analysis


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ GWAS-focused dataset: 100 rows
📊 Unique genes: 9
📚 Unique GWAS studies: 91

📋 Columns: ['leftStudyLocusId', 'rightStudyLocusId', 'chromosome', 'h4', 'studyId', 'targetId', 'l2g_score', 'studyType', 'traitFromSource', 'projectId']

🔬 Top GWAS colocalizations:
  chromosome   h4         targetId       studyId  \
0         19  1.0  ENSG00000142252  GCST90446004   
1          1  1.0  ENSG00000169174  GCST90445989   
2          1  1.0  ENSG00000169174  GCST90445982   
3          1  1.0  ENSG00000169174  GCST90269513   
4          1  1.0  ENSG00000169174  GCST90446134   

                                     traitFromSource  
0  Omega-6 Fatty Acids to Polyunsaturated Fatty A...  
1  Free Cholesterol to Cholesterol in Medium VLDL...  
2  Cholesteryl esters in medium VLDL (UKB data fi...  
3  Cholesteryl esters in VLDL (UKB data field 23416)  
4  Free cholesterol in very small VLDL (UKB data ...  

💊 Final query with drugs:


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Final dataset: 100 rows

📊 Data completeness:
  targetId: 100/100 (100.0%)
  diseaseId: 100/100 (100.0%)
  drugId: 100/100 (100.0%)
  diseaseName: 100/100 (100.0%)
  traitFromSource: 100/100 (100.0%)

🔍 DataFrame structure validation:
✅ leftStudyLocusId
✅ rightStudyLocusId
✅ chromosome
✅ h4
✅ studyId
✅ targetId
✅ diseaseId
✅ drugId
✅ diseaseName

👀 Sample of final DataFrame:
  chromosome   h4         targetId  \
0          1  1.0  ENSG00000169174   
1          1  1.0  ENSG00000169174   
2          1  1.0  ENSG00000169174   
3          1  1.0  ENSG00000169174   
4          1  1.0  ENSG00000169174   

                                     traitFromSource         drugId  \
0   Triglycerides to total lipids ratio in large LDL  CHEMBL2364655   
1  Free cholesterol to total lipids in medium VLD...  CHEMBL4594566   
2  Total esterified cholesterol levels (UKB data ...  CHEMBL2364655   
3                        Linoleic acid (18:2) levels  CHEMBL2364655   
4  Total cholesterol levels in chylo